In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.corpus import stopwords
import nltk
import re


In [8]:

# Jika belum, unduh stopwords untuk nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Load data
# file_path = 'youtube_comments_i6IOiUi6IYY.xlsx'
file_path = 'youtube_comments_labeled_rule_based_advanced.xlsx'
data = pd.read_excel(file_path)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# Pra-pemrosesan Teks
def preprocess_text(text):
    # Ubah teks menjadi lowercase
    text = text.lower()
    # Hilangkan karakter khusus dan angka
    text = re.sub(r'[^a-z\s]', '', text)
    # Hilangkan stop words
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

In [10]:
data['Comment'] = data['Comment'].apply(preprocess_text)

# Membagi data menjadi training dan testing (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(data['Comment'], data['SPAM'], test_size=0.2, random_state=42)

# Ekstraksi Fitur dengan TF-IDF
tfidf = TfidfVectorizer(max_features=5000)  # Membatasi fitur ke 5000 kata paling sering muncul
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Melatih model Logistic Regression
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Prediksi pada data testing
y_pred = model.predict(X_test_tfidf)

# Evaluasi Model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='SPAM')
recall = recall_score(y_test, y_pred, pos_label='SPAM')
f1 = f1_score(y_test, y_pred, pos_label='SPAM')

print(f"Akurasi: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# Menggunakan model untuk melabeli seluruh data
data['Predicted_SPAM'] = model.predict(tfidf.transform(data['Comment']))

# Simpan hasil ke file baru
output_path = 'youtube_comments_ml_labeled.xlsx'
data.to_excel(output_path, index=False)

print(f"File with labeled comments saved as {output_path}")


Akurasi: 0.95
Precision: 0.00
Recall: 0.00
F1 Score: 0.00
File with labeled comments saved as youtube_comments_ml_labeled.xlsx


c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
